In [1]:
from pandas import read_csv, to_datetime, merge, concat
from plotly.express import line

In [2]:
satcat_df = read_csv('../data/celestrak_satcat.csv').drop(0, axis=0)

In [3]:
satcat_df.shape

(60001, 17)

In [4]:
satcat_df.columns

Index(['OBJECT_NAME', 'OBJECT_ID', 'NORAD_CAT_ID', 'OBJECT_TYPE',
       'OPS_STATUS_CODE', 'OWNER', 'LAUNCH_DATE', 'LAUNCH_SITE', 'DECAY_DATE',
       'PERIOD', 'INCLINATION', 'APOGEE', 'PERIGEE', 'RCS', 'DATA_STATUS_CODE',
       'ORBIT_CENTER', 'ORBIT_TYPE'],
      dtype='object')

In [5]:
satcat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60001 entries, 1 to 60001
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECT_NAME       60001 non-null  object 
 1   OBJECT_ID         60001 non-null  object 
 2   NORAD_CAT_ID      60001 non-null  int64  
 3   OBJECT_TYPE       60001 non-null  object 
 4   OPS_STATUS_CODE   42754 non-null  object 
 5   OWNER             60001 non-null  object 
 6   LAUNCH_DATE       60001 non-null  object 
 7   LAUNCH_SITE       60001 non-null  object 
 8   DECAY_DATE        31073 non-null  object 
 9   PERIOD            59110 non-null  float64
 10  INCLINATION       59110 non-null  float64
 11  APOGEE            59110 non-null  float64
 12  PERIGEE           59110 non-null  float64
 13  RCS               32930 non-null  float64
 14  DATA_STATUS_CODE  1058 non-null   object 
 15  ORBIT_CENTER      60001 non-null  object 
 16  ORBIT_TYPE        60001 non-null  object

In [6]:
satcat_df['LAUNCH_DATE'] = to_datetime(satcat_df['LAUNCH_DATE'], errors='coerce')
satcat_df['DECAY_DATE'] = to_datetime(satcat_df['DECAY_DATE'], errors='coerce')

In [7]:
active_status_codes = ['+', 'P', 'B', 'S', 'X']
active_sats = satcat_df[satcat_df['OPS_STATUS_CODE'].isin(active_status_codes)]
inactive_sats = satcat_df[~satcat_df['OPS_STATUS_CODE'].isin(active_status_codes)]

In [8]:
active_sats.shape

(10250, 17)

In [9]:
inactive_sats.shape

(49751, 17)

In [10]:
# Get the total number of launches over time
satcat_df['LAUNCH_YEAR'] = satcat_df['LAUNCH_DATE'].dt.year
satcat_df['LAUNCH_MONTH_YEAR'] = satcat_df['LAUNCH_DATE'].dt.to_period('M').astype(str)
launches_over_time = satcat_df.groupby(['LAUNCH_MONTH_YEAR']).size()
launches_over_time = launches_over_time.reset_index()
launches_over_time = launches_over_time.rename(columns={'LAUNCH_MONTH_YEAR': 'MONTH_YEAR', 0: 'LAUNCH_COUNT'})
launches_over_time['LAUNCH_COUNT_CUM'] = launches_over_time['LAUNCH_COUNT'].cumsum()
launches_over_time

,MONTH_YEAR,LAUNCH_COUNT,LAUNCH_COUNT_CUM
0,1957-10,1,1
1,1957-11,1,2
2,1958-02,1,3
3,1958-03,4,7
4,1958-05,2,9
...,...,...,...
781,2024-02,184,59106
782,2024-03,268,59374
783,2024-04,239,59613
784,2024-05,295,59908


In [11]:
# Get the number of satellites decayed over time
satcat_df['DECAY_MONTH_YEAR'] = satcat_df['DECAY_DATE'].dt.to_period('M').astype(str)
decays_over_time = satcat_df.dropna(subset='DECAY_DATE').groupby(['DECAY_MONTH_YEAR']).size()
decays_over_time = decays_over_time.reset_index()
decays_over_time = decays_over_time.rename(columns={'DECAY_MONTH_YEAR': 'MONTH_YEAR', 0: 'DECAY_COUNT'})
decays_over_time['DECAY_COUNT_CUM'] = decays_over_time['DECAY_COUNT'].cumsum()
decays_over_time

,MONTH_YEAR,DECAY_COUNT,DECAY_COUNT_CUM
0,1958-01,1,1
1,1958-04,1,2
2,1958-06,1,3
3,1958-10,1,4
4,1958-12,2,6
...,...,...,...
775,2024-02,123,30553
776,2024-03,141,30694
777,2024-04,153,30847
778,2024-05,155,31002


In [12]:
# Create a combined dataframe to work with
merged_df = merge(launches_over_time, decays_over_time, on='MONTH_YEAR', how='outer').fillna(0)
merged_df['ON_ORBIT'] = (merged_df['LAUNCH_COUNT'] - merged_df['DECAY_COUNT']).cumsum()
on_orbit = merged_df[['MONTH_YEAR', 'ON_ORBIT']]
merged_df

,MONTH_YEAR,LAUNCH_COUNT,LAUNCH_COUNT_CUM,DECAY_COUNT,DECAY_COUNT_CUM,ON_ORBIT
0,1957-10,1.0,1.0,0.0,0.0,1.0
1,1957-11,1.0,2.0,0.0,0.0,2.0
2,1958-01,0.0,0.0,1.0,1.0,1.0
3,1958-02,1.0,3.0,0.0,0.0,2.0
4,1958-03,4.0,7.0,0.0,0.0,6.0
...,...,...,...,...,...,...
785,2024-02,184.0,59106.0,123.0,30553.0,28553.0
786,2024-03,268.0,59374.0,141.0,30694.0,28680.0
787,2024-04,239.0,59613.0,153.0,30847.0,28766.0
788,2024-05,295.0,59908.0,155.0,31002.0,28906.0


In [13]:
# Rename columns for legend readability
plot_df = merged_df.rename(columns={
    'ON_ORBIT': 'On Orbit',
    'LAUNCH_COUNT_CUM': 'Launches',
    'DECAY_COUNT_CUM': 'Decayed Satellites'
})

In [14]:
fig = line(
    plot_df,
    x='MONTH_YEAR',
    y=['On Orbit', 'Launches', 'Decayed Satellites'],
    labels={
        'value': 'Count',
        'MONTH_YEAR': 'Year'
    },
    title='Satellite Growth Over Time',
    color_discrete_sequence=['#2c57c9', '#8d50d0', '#c95574']
)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), legend_title=None)
fig.update_traces(line={'width': 3})
fig.show()

In [15]:
# Get Starlink launches per year
starlink_satcat_df = satcat_df[(satcat_df['OBJECT_NAME'].str.contains('STARLINK', na=False)) & (satcat_df['LAUNCH_YEAR'] > 2019)]
starlink_launches = starlink_satcat_df.groupby('LAUNCH_MONTH_YEAR').size().cumsum().reset_index(name='LAUNCHES')
starlink_launches['Type'] = 'Starlink'

# Get other launches per year
other_satcat_df = satcat_df[(~satcat_df['OBJECT_NAME'].str.contains('STARLINK', na=False)) & (satcat_df['LAUNCH_YEAR'] > 2019)]
other_launches = other_satcat_df.groupby('LAUNCH_MONTH_YEAR').size().cumsum().reset_index(name='LAUNCHES')
other_launches['Type'] = 'Other'

# Combine
combined_launches = concat([starlink_launches, other_launches])
combined_launches

,LAUNCH_MONTH_YEAR,LAUNCHES,Type
0,2020-01,120,Starlink
1,2020-02,180,Starlink
2,2020-03,240,Starlink
3,2020-04,300,Starlink
4,2020-06,418,Starlink
...,...,...,...
49,2024-02,4733,Other
50,2024-03,4820,Other
51,2024-04,4855,Other
52,2024-05,4926,Other


In [16]:
fig = line(
    combined_launches,
    x='LAUNCH_MONTH_YEAR',
    y='LAUNCHES',
    color='Type',
    labels={
        'LAUNCH_MONTH_YEAR': 'Launch Year',
        'LAUNCHES': 'Number of Launches'
    },
    title='Number of Starlink vs All Other Satellite Launches',
    color_discrete_sequence=['#2c57c9', '#c95574']
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Launches',legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), legend_title=None
)
fig.update_traces(line={'width': 4})

fig.show()